# TCC

In [3]:
import pandas as pd
import os as os
import PyPDF2
import fitz

import re

## Mapeando questões das provas

In [28]:
def map_questions(exam_string):    
    
    # Regex para o split usando "questão", espaço em branco e número (case-insensitive)
    pattern = re.compile(r'(questão\s+\d+)', re.IGNORECASE)
    result = pattern.split(exam_string)
    
    # Remover strings vazias do resultado
    return [item.strip() for item in result if item.strip()]

In [5]:
def separate_question(question):
        
    # Expressão regular para encontrar o corpo e as alternativas
    # pattern = re.compile(r'(\nA\s.*?\nB\s.*?\nC\s.*?\nD\s.*?\nE\s.*?\.)', question, re.DOTALL)
    # print(pattern)
    # print(pattern.findall(question))

    # Encontrar a correspondência na string
    alternatives_regex = r'\nA\s.*?\nB\s.*?\nC\s.*?\nD\s.*?\nE\s.*?\.'
    body = re.search(f'^(.*?)(?={alternatives_regex})', question, re.DOTALL).group(1)
    alternatives = re.search(f'({alternatives_regex})', question, re.DOTALL).group(1)    
    
    return ({'body': body, 'alternatives':  alternatives})


In [17]:
testando_regex = """O termo agronegócio não se refere apenas à agricultura 
e à pecuária, pois as atividades ligadas a essa produção 
incluem fornecedores de equipamentos, serviços para a 
zona rural, industrialização e comercialização dos produtos.
Os dados seguintes refletem a participação percentual 
do agronegócio no PIB brasileiro entre os anos de 1998 
e 2008:
• 
1998: 21,33%;
• 
1999: 22,24%;
• 
2000: 22,87%;
• 
2001: 23,26%;
• 
2002: 25,31%;
• 
2003: 28,28%;
• 
2004: 27,79%;
• 
2005: 25,83%;
• 
2006: 23,92%;
• 
2007: 24,74%;
• 
2008: 26,46%.
Esses dados foram citados em uma palestra na 
qual o orador ressaltou uma queda da participação do 
agronegócio no PIB brasileiro e a posterior recuperação 
dessa participação, em termos percentuais.
De acordo com essas informações, o período de queda 
ocorreu entre os anos de 
A 1998 e 2001.
B 2001 e 2003.
C 2003 e 2006.
D 2003 e 2007.
E 2003 e 2008."""

testando_regex = """QuESTÃO 160 
O prefeito de uma cidade deseja construir uma 
rodovia para dar acesso a outro município. Para isso, foi 
aberta uma licitação na qual concorreram duas empresas. A primeira cobrou R$ 100 000,00 por km construído (n), 
acrescidos de um valor fixo de R$ 350 000,00, enquanto 
a segunda cobrou R$ 120 000,00 por km construído (n), 
acrescidos de um valor fixo de R$ 150 000,00. As duas 
empresas apresentam o mesmo padrão de qualidade 
dos serviços prestados, mas apenas uma delas poderá 
ser contratada.
Do ponto de vista econômico, qual equação possibilitaria 
encontrar a extensão da rodovia que tornaria indiferente 
para a prefeitura escolher qualquer uma das propostas 
apresentadas?
A 100n + 350 = 120n + 150 
B 100n + 150 = 120n + 350   
C 100(n + 350) = 120(n + 150) 
D 100(n + 350 000) = 120(n + 150 000) 
E 350(n + 100 000) = 150(n + 120 000)."""

print(separate_question(testando_regex))

{'body': 'QuESTÃO 160 \nO prefeito de uma cidade deseja construir uma \nrodovia para dar acesso a outro município. Para isso, foi \naberta uma licitação na qual concorreram duas empresas. A primeira cobrou R$ 100 000,00 por km construído (n), \nacrescidos de um valor fixo de R$ 350 000,00, enquanto \na segunda cobrou R$ 120 000,00 por km construído (n), \nacrescidos de um valor fixo de R$ 150 000,00. As duas \nempresas apresentam o mesmo padrão de qualidade \ndos serviços prestados, mas apenas uma delas poderá \nser contratada.\nDo ponto de vista econômico, qual equação possibilitaria \nencontrar a extensão da rodovia que tornaria indiferente \npara a prefeitura escolher qualquer uma das propostas \napresentadas?', 'alternatives': '\nA 100n + 350 = 120n + 150 \nB 100n + 150 = 120n + 350   \nC 100(n + 350) = 120(n + 150) \nD 100(n + 350 000) = 120(n + 150 000) \nE 350(n + 100 000) = 150(n + 120 000).'}


### Convertendo PDF em string

In [ ]:
 
doc = fitz.open('../TCC/pdfs/2011/2011_D2_CD6_cinza.pdf') 
text = "" 
splitter = ""
for page in doc: 
    if ("\nQuESTÃO" in page.get_text()):
        text+=page.get_text() 
    elif ("\nQUESTÃO" in page.get_text()):
        text+=page.get_text() 
    elif ("Questão" in page.get_text()):
        text+=page.get_text()     
        
teste = map_questions(text)

print(len(teste))
for i in range(len(teste)):
    print(teste[i])
    print("-------------------------------------------------")
    print("Questão separada")
    print("-------------------------------------------------")
    print(separate_question(teste[i]))
    print("-------------------------------------------------")
    print("-------------------------------------------------")

# string_teste = "Going to university seems to reduce the risk of dying from coronary heart disease. An American study that involved 10 000 patients from around the world has found that people who leave school before the age of 16 are five times more likely to suffer a heart attack and die than university graduates. Em relação às pesquisas, a utilização da expressão university graduates evidencia a intenção de informar que A as doenças do coração  atacam dez mil pacientes. B as doenças do coração ocorrem na faixa dos dezesseis anos. C as pesquisas sobre doenças são divulgadas no meio acadêmico. D jovens americanos são alertados dos riscos de doenças do coração. E maior nível de estudo reduz riscos de ataques do coração."
# print(separate_question(string_teste))

In [7]:
PATH = "../TCC/pdfs"
years = os.listdir(f'{PATH}')
for year in years:
    
    exams = os.listdir(f'{PATH}/{year}')
    all_questions_edition = ''
    for test in exams:

        text = "" 
        if( "_D2_" in test):
            text += "\n------D2------\n"
                
        doc = fitz.open(f'{PATH}/{year}/{test}') 
        
        for page in doc: 
            if ("\nQuESTÃO" in page.get_text()):
                text += page.get_text() 
            elif ("\nQUESTÃO" in page.get_text()):
                text += page.get_text() 
            elif ("Questão" in page.get_text()):
                text += page.get_text()         
        all_questions_edition += text
       
    with open(f'{PATH}/{year}/prova{year}.txt', 'w', encoding='utf-8') as exam_str:
        exam_str.write(all_questions_edition)
        
    break


### Lendo os arquivos com os textos das provas e mapeando as questões em um Dataframe

In [8]:
def agg_question_info(question, year,number):
    
    if(int(year) > 2017):
        if(number <= 45):
            area = 'CH - Ciências Humanas'
        elif(number > 45 and number <= 90):
            area = 'CN - Ciências da Natureza'
        elif(number > 90 and number <= 140):
            if(number > 90 and number <= 95):
                sp_area = 'Língua estrangeira - Inglês'
            elif(number > 95 and number <= 100):
                sp_area = 'Língua estrangeira - Espanhol'
            area = 'LC - Linguagens e Códigos'
        elif(number > 140 and number <= 185):
            area = 'MT - Matemática e Códigos'
    else:
        if(number <= 45):
            area = ''
        elif(number > 45 and number <= 90):
            area = ''
        elif(number > 90 and number <= 140):
            if(number > 90 and number <= 95):
                sp_area = 'Língua estrangeira - Inglês'
            elif(number > 95 and number <= 100):
                sp_area = 'Língua estrangeira - Espanhol'
            area = ''
        elif(number > 140 and number <= 185):
            area = ''
        
    return pd.Series({
        'id': number,
        'year': year,
        'area': area,
        'sp_area': sp_area,
        'body': question['body'],
        'alternatives': question['alternatives']
    })

In [31]:
enem_questions = pd.DataFrame(columns=['id','year','area','sp_area' ,'body','alternatives','result','answer'])
TXT_PATH = "../TCC/tests_txt_files"

for year in years:
    
    test = os.listdir(f'{TXT_PATH}/{year}')

    with open(f'{TXT_PATH}/{year}/prova{year}.txt', 'r', encoding='utf-8') as exam_str:
        
        exam_string = exam_str.read()
        questions = map_questions(exam_string)
        counter = 0
        print(questions)
        for question in questions:
            print(question)
            
            if(('Questão' in question) or ('QUESTÃO' in question) or ('QuESTÃO' in question)):
                # separar corpo e alternativas
                counter += 1
                print(question)
                question_obj = separate_question(question)
                question_info = agg_question_info(question_obj, year, counter)
            
                # enem_questions = enem_questions.append(question_info, ignore_index=True)
                enem_questions = pd.DataFrame.concat([enem_questions, question_info], ignore_index=True)

    enem_questions.to_csv(f'../TCC/csvs/enem_{year}_questions.csv', index=False)
    
            # print("-------------------------------------------------")
            # print("-------------------------------------------------")
            # print("-------------------------------------------------")
            


['*LEBR75SAB1*\nCH - 1º dia | Caderno 3 - BRANCO - Página 1\nCIÊNCIAS HUMANAS E SUAS \nTECNOLOGIAS\nQuestões de 1 a 45', 'QUESTÃO 01', 'No mundo árabe, países governados há décadas por \nregimes políticos centralizadores contabilizam metade \nda população com menos de 30 anos; desses, 56% \ntêm acesso à internet. Sentindo-se sem perspectivas \nde futuro e diante da estagnação da economia, esses \njovens incubam vírus sedentos por modernidade e \ndemocracia. Em meados de dezembro, um tunisiano \nde 26 anos, vendedor de frutas, põe fogo no próprio \ncorpo em protesto por trabalho, justiça e liberdade. Uma \nsérie de manifestações eclode na Tunísia e, como uma \nepidemia, o vírus libertário começa a se espalhar pelos \npaíses vizinhos, derrubando em seguida o presidente \ndo Egito, Hosni Mubarak.  Sites e redes sociais – \ncomo o Facebook e o Twitter – ajudaram a mobilizar \nmanifestantes do norte da África a ilhas do Golfo Pérsico.\nConsiderando os movimentos políticos mencionados no \nt

AttributeError: 'NoneType' object has no attribute 'group'

In [40]:
teste_split= 'questão 1 abacate questão 2 QUEstão 3'
teste = teste_split.split(r'questão\s+\d+', re.IGNORECASE)
teste1 = teste_split.split(r'questão\s', re.IGNORECASE)
print(teste)
print(teste1)

['questão 1 abacate questão 2 QUEstão 3']
['questão 1 abacate questão 2 QUEstão 3']
